# OpenAI Compatible API

SGLang provides an OpenAI compatible API for smooth transition from OpenAI services.

- `chat/completions`
- `completions`
- `batch`
- `embeddings`(refer to [embedding_model.ipynb](embedding_model.ipynb))

## Chat Completions

### Usage

Similar to [send_request.ipynb](send_request.ipynb), we can send a chat completion request to SGLang server with OpenAI API format.

In [1]:
from sglang.utils import execute_shell_command, wait_for_server, terminate_process


server_process = execute_shell_command("""
python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
--port 30000 --host 0.0.0.0 --log-level warning
""")

wait_for_server("http://localhost:30000")
print("Server is ready. Proceeding with the next steps.")

Server is ready. Proceeding with the next steps.


In [ ]:
import openai

# Always assign an api_key, even if not specified during server initialization.
# Setting an API key during server initialization is strongly recommended.

client = openai.Client(
    base_url="http://127.0.0.1:30000/v1", api_key="None"
)

# Chat completion example

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant"},
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)
print(response)

### Parameters

The chat completions API accepts the following parameters:

- `messages`: List of messages in the conversation, each containing `role` and `content`
- `model`: The model identifier to use for completion
- `max_tokens`: Maximum number of tokens to generate in the response
- `temperature`: Controls randomness (0-2). Lower values make output more focused and deterministic
- `top_p`: Alternative to temperature. Controls diversity via nucleus sampling
- `n`: Number of chat completion choices to generate
- `stream`: If true, partial message deltas will be sent as they become available
- `stop`: Sequences where the API will stop generating further tokens
- `presence_penalty`: Penalizes new tokens based on their presence in the text so far (-2.0 to 2.0)
- `frequency_penalty`: Penalizes new tokens based on their frequency in the text so far (-2.0 to 2.0)
- `logit_bias`: Modify the likelihood of specified tokens appearing in the completion
- `logprobs`: Include log probabilities of tokens in the response
- `top_logprobs`: Number of most likely tokens to return probabilities for
- `seed`: Random seed for deterministic results
- `response_format`: Specify the format of the response (e.g., JSON)
- `stream_options`: Additional options for streaming responses
- `user`: A unique identifier representing your end-user

Here is an example of a detailed chat completion request:

In [ ]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a knowledgeable historian who provides concise responses."},
        {"role": "user", "content": "Tell me about ancient Rome"},
        {"role": "assistant", "content": "Ancient Rome was a civilization centered in Italy."},
        {"role": "user", "content": "What were their major achievements?"}
    ],
    temperature=0.3,                    # Lower temperature for more focused responses
    max_tokens=100,                     # Reasonable length for a concise response
    top_p=0.95,                        # Slightly higher for better fluency
    stop=["\n\n"],                     # Simple stop sequence
    presence_penalty=0.2,               # Mild penalty to avoid repetition
    frequency_penalty=0.2,              # Mild penalty for more natural language
    n=1,                               # Single response is usually more stable
    seed=42,                           # Keep for reproducibility
    stream=True                        # Keep streaming for real-time output
)

for chunk in response:
    print(chunk.choices[0].delta.content or "", end="")

## Completions

### Usage

Completions API is similar to Chat Completions API, but without the `messages` parameter.

In [ ]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None
)
print(response)

### Parameters

The completions API accepts the following parameters:

- `model`: The model identifier to use for completion
- `prompt`: Input text to generate completions for. Can be a string, array of strings, or token arrays
- `best_of`: Number of completions to generate server-side and return the best one
- `echo`: If true, the prompt will be included in the response
- `frequency_penalty`: Penalizes new tokens based on their frequency in the text so far (-2.0 to 2.0)
- `logit_bias`: Modify the likelihood of specified tokens appearing in the completion
- `logprobs`: Include log probabilities of tokens in the response
- `max_tokens`: Maximum number of tokens to generate in the response (default: 16)
- `n`: Number of completion choices to generate
- `presence_penalty`: Penalizes new tokens based on their presence in the text so far (-2.0 to 2.0)
- `seed`: Random seed for deterministic results
- `stop`: Sequences where the API will stop generating further tokens
- `stream`: If true, partial completion deltas will be sent as they become available
- `stream_options`: Additional options for streaming responses
- `suffix`: Text to append to the completion
- `temperature`: Controls randomness (0-2). Lower values make output more focused and deterministic
- `top_p`: Alternative to temperature. Controls diversity via nucleus sampling
- `user`: A unique identifier representing your end-user

Here is an example of a detailed completions request:

In [ ]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,                    # Moderate temperature for creative writing
    max_tokens=150,                     # Longer response for a story
    top_p=0.9,                         # Balanced diversity in word choice
    stop=["\n\n", "THE END"],          # Multiple stop sequences
    presence_penalty=0.3,               # Encourage novel elements
    frequency_penalty=0.3,              # Reduce repetitive phrases
    n=1,                               # Generate one completion
    seed=123,                          # For reproducible results
    stream=True                        # Stream the response
)

for chunk in response:
    print(chunk.choices[0].text or "", end="")

## Batch

We have implemented the batch API for chat completions and completions. You can upload your requests in JSONL format, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batch APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`


In [12]:
import json
from openai import OpenAI

client = OpenAI(
    base_url="http://127.0.0.1:30000/v1",
    api_key="None"
)

requests = [
    {
        "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
        "messages": [
            {"role": "user", "content": "Tell me a joke about programming"}
        ]
    },
    {
        "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
        "messages": [
            {"role": "user", "content": "What is Python?"}
        ]
    }
]

with open('batch_requests.jsonl', 'w') as f:
    for req in requests:
        f.write(json.dumps(req) + '\n')

with open('batch_requests.jsonl', 'rb') as f:
    file_response = client.files.create(
        file=f,
        purpose='batch'
    )

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint='/chat/completions',
    completion_window='24h'
)

print(f"Batch job created with ID: {batch_response.id}")
batch_status = client.batches.retrieve(batch_id=batch_response.id)

Batch job created with ID: batch_02b2636f-06a7-4982-b18e-7f6c954cc5af


It takes a while to complete the batch job. You can use these two APIs to cancel the batch job or retrieve the batch job status.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

In [11]:
batch_status = client.batches.retrieve(batch_id=batch_response.id)
print(f"Current status: {batch_status.status}")

Current status: failed


In [22]:
if batch_status.status == "running":
    cancelled_response = client.batches.cancel(batch_id=batch_response.id)